In [10]:
from __future__   import division
import numpy      as np
import array
import math
import struct

def read_mnist(images_file, labels_file): 
    f1 = open(labels_file, 'rb')
    magic_number, size = struct.unpack(">II", f1.read(8))
    labels = array.array("b", f1.read())
    f1.close()
    
    f2 = open(images_file, 'rb')
    magic_number, size, rows, cols = struct.unpack(">IIII", f2.read(16))
    raw_images = array.array("B", f2.read())
    f2.close()

    N = len(labels)
    images = np.zeros((N, rows*cols), dtype=np.uint8)
    for i in range(N):
        images[i] = np.array(raw_images[ i*rows*cols : (i+1)*rows*cols ])

    return images, labels

In [11]:
# Read Training data.
TRAIN_IMAGES  = "C:\\Users\\oop\\Desktop\\Winter 2016\\train-images.idx3-ubyte"
TRAIN_LABELS  = "C:\\Users\\oop\\Desktop\\Winter 2016\\train-labels.idx1-ubyte"
images_train, labels_train = read_mnist(TRAIN_IMAGES, TRAIN_LABELS)
images_train, labels_train = images_train[:20000], labels_train[:20000]

# Read Test data.
TEST_IMAGES = "C:\\Users\\oop\\Desktop\\Winter 2016\\t10k-images.idx3-ubyte"
TEST_LABELS  = "C:\\Users\\oop\\Desktop\\Winter 2016\\t10k-labels.idx1-ubyte"
images_test, labels_test = read_mnist(TEST_IMAGES, TEST_LABELS)
images_test, labels_test = images_test[:2000], labels_test[:2000]

In [12]:
sigmoid = lambda x : 1/(1+math.exp(-x))

In [13]:
sigmoid_vector_fn = np.vectorize(sigmoid)
def get_gradient(X, T, weight_vector):
    Y = sigmoid_vector_fn(np.dot(X, weight_vector))
    return np.dot(X.transpose(), T-Y)

In [14]:
def logistic_regression(X, T):
    weight_vector = np.zeros(len(X[0]))
    learning_rate = 0.000002/len(X)
    for i in range(400):
        weight_vector += learning_rate * get_gradient(X, T, weight_vector)
    return weight_vector

In [15]:
def train():
    X = np.array([np.insert(x, 0, 1) for x in images_train])
    weights = []
    for digit in range(10):
        T = [1 if label == digit else 0 for label in labels_train]
        weights.append(logistic_regression(X, T))
    return np.array(weights)

In [16]:
def test():
    weights = train()
    X = np.array([np.insert(x, 0, 1) for x in images_test])
    
    ############ Accuracy calculations for each 10 2-way classifications #############
    Y_probs = []
    for digit in range(10):
        T = np.array([1 if label == digit else 0 for label in labels_test])
        Y_prob = sigmoid_vector_fn(X.dot(weights[digit]))
        Y_probs.append(Y_prob)
        Y = np.array([1 if Y_prob[i] >= 0.5 else 0 for i in range(len(X))])
        correct_count = sum([Y[i] == T[i] for i in range(len(Y))])
        print "Accuracy for digit {" + str(digit) + "} classification is " + \
              str(correct_count * 100.0 / len(X))
    
    ############ Accuracy calculations for each 10 2-way classifications #############
    Y_probs = np.array(Y_probs).transpose()
    correct_count = sum([prob_list.argmax() == labels_test[i] 
                         for i, prob_list in enumerate(Y_probs)])
    print "Total accuracy is " + str(correct_count * 100.0 / len(X))

In [17]:
test()

Accuracy for digit {0} classification is 98.6
Accuracy for digit {1} classification is 98.75
Accuracy for digit {2} classification is 96.65
Accuracy for digit {3} classification is 96.6
Accuracy for digit {4} classification is 96.95
Accuracy for digit {5} classification is 96.1
Accuracy for digit {6} classification is 97.1
Accuracy for digit {7} classification is 96.75
Accuracy for digit {8} classification is 92.7
Accuracy for digit {9} classification is 95.1
Total accuracy is 86.05
